# Assignment 2 
### COMP 551 
### Olivier Simard-Morissette 
### 260563480 

## Q1

In [211]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplot
import seaborn as sns
import math
sns.set_palette(sns.color_palette("GnBu_d"))
%matplotlib inline




In [212]:
def get_mean_from_file(file_path):
    '''
    Opens the file provided in the instructor to retrieve the means
    for the different distributions.
    '''
    with open(file_path) as file_handler:
        # strips out last line because of extra comma in file.
        list_of_means = file_handler.read().strip().split(',')[:-1] 
        list_of_means_float = [np.float(mean) for mean in list_of_means]
        array_of_means = np.array(list_of_means_float)
#         vector_of_means = array_of_means.reshape(len(array_of_means),1)
        return array_of_means
 
def get_covariance_matrix_from_file(file_path):
    # strips out last column because of extra commas. 
    df_covariance = pd.read_csv(file_path, header=None)
    del df_covariance[20]
    return df_covariance.as_matrix()



m0 = get_mean_from_file('./hwk2_datasets_corrected/DS1_m_0.txt')
m1 = get_mean_from_file('./hwk2_datasets_corrected/DS1_m_1.txt')
covariance_matrix = get_covariance_matrix_from_file('./hwk2_datasets_corrected/DS1_Cov.txt')

m0_dataset = np.random.multivariate_normal(m0, covariance_matrix, size=2000)
m1_dataset = np.random.multivariate_normal(m1, covariance_matrix, size=2000)

# Labeling the data on the 20th column with 0 corresponding to 
# a negative example and 1 corresponding to a positive example.
df_m0 = pd.DataFrame(m0_dataset)
df_m0[20] = 0

df_m1 = pd.DataFrame(m1_dataset)
df_m1[20] = 1

# Concatenating the two datasets into a single dataframe.
df = pd.concat([df_m1, df_m0],ignore_index=True)
df = df.reindex(np.random.permutation(df.index))

test_df = df[:1200].reset_index(drop=True)
train_df = df[1200:].reset_index(drop=True)

test_df.to_csv('./my_dataset/DS1_test.csv')
train_df.to_csv('./my_dataset/DS1_train.csv')



In [251]:
def get_column_vector_means(df, class_num):
    train_matrix_means = [] 
    class_df = df[df[20] == class_num]
    for column in class_df:
        if column != 20:
            train_matrix_means.append(class_df[column].mean())
    return np.asarray(train_matrix_means)

def get_covariance_sum(df, class_num, mean_vector):
    count = 0 
    class_df = df[df[20] == class_num] # Only get the sum for this class. 
    class_df.drop([20], axis = 1, inplace = True) # Remove the class label 
    class_df_as_matrix = class_df.as_matrix()
    
    # Execute summation 
    covariance_running_sum = np.zeros((20,20))
    for i in range(len(class_df_as_matrix)):
        mean_difference = class_df_as_matrix[i] - mean_vector
        mean_diff_vector = mean_difference.reshape(len(mean_difference),1)
        mean_diff_vector_transpose = mean_diff_vector.transpose()
        covariance_component = np.divide(np.matmul(mean_diff_vector,mean_diff_vector_transpose),len(df))
        covariance_running_sum = np.add(covariance_running_sum, covariance_component )

    return covariance_running_sum
    
    
    


class_nums = (0, 1)
class_mean_vectors = {}
for class_num in class_nums:
    class_mean_vector = get_column_vector_means(train_df, class_num)
    class_mean_vectors[class_num] = class_mean_vector
    
covariance = np.zeros((20,20))
for class_num in class_nums:
    mean_vector = class_mean_vectors[class_num]
    covariance_sum = get_covariance_sum(train_df, class_num, mean_vector)
    covariance = np.add(covariance, covariance_sum)

class_probability = {}
for class_num in class_nums:
    class_probability[class_num] = len(train_df[train_df[20] == class_num]) / float(len(train_df))



def decision_boundary(x,class_probability_lookup, class_mean_vector_lookup, covariance):
    '''
    Takes a test sample x as input and returns true if belonging to class zero,
    otherwise returns false 
    '''
    prob_zero = class_probability_lookup[0]
    prob_one = class_probability_lookup[1]
    covariance_inverse = np.linalg.inv(covariance)
    mv_zero = class_mean_vector_lookup[0]
    mv_zero = mv_zero.reshape(len(mv_zero), 1)
    mv_one = class_mean_vector_lookup[1]
    mv_one = mv_one.reshape(len(mv_one), 1)
    mv_zero_transpose = mv_zero.transpose()
    mv_one_transpose = mv_one.transpose()
    x_transpose = x.transpose()
    
    xtw1 = np.matmul(np.matmul(x_transpose,covariance_inverse),np.subtract(mv_zero,mv_one))
    x0_term1 = math.log(prob_zero) - math.log(prob_one)
    x0_term2 = -0.5 * np.matmul(np.matmul(mv_zero_transpose,covariance_inverse),mv_zero)
    x0_term3 = 0.5 * np.matmul(np.matmul(mv_one_transpose, covariance_inverse),mv_one)
    sum_decision = (x0_term1 + x0_term2[0][0] + x0_term3[0][0] +  xtw1[0][0])
    return sum_decision > 0 


test_vector = test_df.iloc[40,0:20].as_matrix().reshape(20,1)

print(test_df.iloc[40,20])


decision_boundary(test_vector, class_probability, class_mean_vectors, covariance)





0


/usr/local/lib/python2.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


True